# Data Cleaning and Transformation

In [1]:
import pandas as pd
df = pd.read_csv('/content/music_streaming.csv')

## a) Perform any necessary data cleaning & engineering that renders your data useable (i.e. handling missing values, duplicates, classification, transformation...etc.)

We first check which columns has null values

In [2]:
percent_missing = df.isnull().sum() * 100 / len(df)
percent_missing 

Artist Name            0.000000
Track Name             0.000000
Popularity             2.412897
danceability           0.000000
energy                 0.000000
key                   11.066043
loudness               0.000000
mode                   0.000000
speechiness            0.000000
acousticness           0.000000
instrumentalness      22.766511
liveness               0.000000
valence                0.000000
tempo                  0.010400
duration_in min/ms     0.010400
time_signature         0.010400
Genre                  0.010400
dtype: float64

Then we remove dublicates

In [3]:
df = df.drop_duplicates()

Then we remove the key and instrumentalness columns as they have alot of null values which if predicted won't give an accurate result

In [4]:
df.drop(['key',"instrumentalness"],axis=1,inplace=True)

Now we will fill the null values in the popularity column using the mean of the popularity for each artist

In [5]:
df["Popularity"] = df.groupby("Artist Name").Popularity.transform(lambda x: x.fillna(x.mean()))

In [6]:
percent_missing = df.isnull().sum() * 100 / len(df)
percent_missing 

Artist Name           0.000000
Track Name            0.000000
Popularity            1.185647
danceability          0.000000
energy                0.000000
loudness              0.000000
mode                  0.000000
speechiness           0.000000
acousticness          0.000000
liveness              0.000000
valence               0.000000
tempo                 0.010400
duration_in min/ms    0.010400
time_signature        0.010400
Genre                 0.010400
dtype: float64

As we can see not all the null values were filled as some artists didn't have any values only null values so we couldn't get the mean therefore we will remove these records as they can't be predicted

In [7]:
df[df['Artist Name'] == 'Lilly Wood and The Prick']

,Artist Name,Track Name,Popularity,danceability,energy,loudness,mode,speechiness,acousticness,liveness,valence,tempo,duration_in min/ms,time_signature,Genre
36,Lilly Wood and The Prick,A Song,NaN,0.618,0.438,-7.495,1,0.0762,0.407,0.5,0.651,155.014,209307.0,4.0,6.0


In [8]:
df = df.dropna()

Now all our data is clean and doesn't have any dublicates or null values

In [9]:
percent_missing = df.isnull().sum() * 100 / len(df)
percent_missing 

Artist Name           0.0
Track Name            0.0
Popularity            0.0
danceability          0.0
energy                0.0
loudness              0.0
mode                  0.0
speechiness           0.0
acousticness          0.0
liveness              0.0
valence               0.0
tempo                 0.0
duration_in min/ms    0.0
time_signature        0.0
Genre                 0.0
dtype: float64

* Now we will convert the milisecond values in duration_in min/ms column to minutes

* We will convert each value that is greater than 10 to minutes by dividing it by 60000 and add it to a new column called duration_in_min

* Then we will remove any value that is greater than 10 minutes or samller than 30 seconds

* Then we drop the duration_in min/ms column as we don't need it anymore

In [10]:
duration_in_min = []

for row in df['duration_in min/ms']:
  if(row > 10):
    duration_in_min.append(row/60000)
  else:
    duration_in_min.append(row)

df['duration_in_min'] = duration_in_min


In [11]:
df = df[ (df['duration_in_min']< 10) | (df['duration_in_min'] > 0.5) ]

In [12]:
df.drop('duration_in min/ms',axis=1,inplace=True)

Now we will rename the columns that have spaces in there names to be able to use them in SQL

In [14]:
df.rename(columns={'Artist Name': 'Artist_Name',
                   'Track Name': 'Track_Name',},
          inplace=True, errors='raise')

Now we will export our dataframe to use it in the other 2 notebooks for SparkSQL and SparkDataframe

In [15]:
df.to_csv('music_transformed.csv')